<img width="8%" alt="Naas.png" src="https://raw.githubusercontent.com/jupyter-naas/awesome-notebooks/master/.github/assets/logos/Naas.png" style="border-radius: 15%">

# Setup

<div class="alert alert-success" role="info">
    <p><b>Note:</b> this data product framework is developed by Naas open source community. You can <a href="https://github.com/sponsors/jupyter-naas" target="_blank">sponsor us</a> if you find it usefull.
</p>
</div>

This notebook allows you to configure and run your data product on your Naas server. *Create an account on [Naas.ai](https://www.naas.ai/free-forever) if you don't already have one.*

Please follow the step below to setup this data product.

## Input

### Pull repository

In [ ]:
!git pull

### Install requirements
Install the packages required to run the product.

In [ ]:
!pip install --user --upgrade -r "requirements.txt"

### Import libraries

In [ ]:
import naas_data_product
import naas
from naas_drivers import gsheet, linkedin
import glob
import os
from IPython.display import clear_output
from tqdm import tqdm

### Setup variables
**Pre-requisite**
- Duplicate our Google Sheets spreadsheet model
- Share your Google Sheets spreadsheet with our service account : 🔗 naas-share@naas-gsheets.iam.gserviceaccount.com

**Inputs**
- `abi_spreadsheet`: Google Sheets spreadsheet URL
- `sheet_entity`: Entity sheet name that stores all your personal data

**Outputs**
- `output_dir_plugin`: Cookie used to authenticate Members and API clients.

In [ ]:
# Inputs
datalake_dir = os.path.join("/", "home", "ftp", "abi", "outputs")
abi_spreadsheet = naas.secret.get("ABI_SPREADSHEET")
force_update = False
if abi_spreadsheet is None or force_update:
    abi_spreadsheet = input("Enter your Google Sheets URL:")
    naas.secret.add("ABI_SPREADSHEET", abi_spreadsheet)
sheet_entity = "ENTITY"

# Outputs
output_dir = os.path.join("outputs", "entity")
output_dir_plugin = os.path.join("outputs", "plugins")

## Model

### Get Entity data from Google Sheet spreadsheet

In [ ]:
df_gsheet = gsheet.connect(abi_spreadsheet).get(sheet_name=sheet_entity).fillna("NA")
df_gsheet.head(1)

### Setup Dependencies

#### Store Entity name

In [ ]:
entity_name = df_gsheet.loc[0, "ENTITY"]
print("Entity name:", entity_name)
pdump(output_dir, entity_name, "entity_name")
naas.dependency.add(os.path.join(output_dir, "entity_name.pickle"))

#### Store Emails
Emails to be used for notifications

In [ ]:
emails = df_gsheet.loc[0, "EMAILS"]
print("- Emails:", emails)
print()
pdump(output_dir, emails, "emails")
naas.dependency.add(os.path.join(output_dir, "emails.pickle"))

#### Store LinkedIn URL

In [ ]:
linkedin_url = df_gsheet.loc[0, "LINKEDIN_URL"]
print("- LinkedIn URL:", linkedin_url)
print()
pdump(output_dir, linkedin_url, "linkedin_url")
naas.dependency.add(os.path.join(output_dir, "linkedin_url.pickle"))

### Setup Naas

#### Add LinkedIn Cookies to Naas secret

In [ ]:
for x in ["LINKEDIN_LI_AT", "LINKEDIN_JSESSIONID"]:
    value = df_gsheet.loc[0, x]
    if value != "NA":
        print(f"Secret '{x}' to be added:")
        naas.secret.add(x, value)

#### Add Naas Chat long lived token to Naas secret

In [ ]:
long_lived_token = naas.secret.get('NAAS_API_TOKEN')
if not long_lived_token:
    long_lived_token = get_long_lived_token(jup_token)
    if long_lived_token:
        naas.secret.add('NAAS_API_TOKEN', long_lived_token)

### Set timezone from profile region and country

In [ ]:
# Get region and country data
linkedin_dir = os.path.join(datalake_dir, "datalake", "linkedin", "profiles")

df = get_linkedin_data(linkedin_url, linkedin_dir, "identity")
region = df.loc[0, "REGION"]
country = df.loc[0, "COUNTRY"]
print("- Region:", region)
print("- Country:", country)

timezone = create_chat_completion(
    long_lived_token,
    prompt="Find timezone in the format 'Region/City'. If there is no exact match, please return a subjective answer based on the data you received",
    message=f"Region: {region}, Country: {country}",
)
print("- Timezone:", timezone)
print()
naas.set_remote_timezone(timezone)
pdump(output_dir, timezone, "timezone")
naas.dependency.add(os.path.join(output_dir, "timezone.pickle"))

## Output

### Run pipelines

In [ ]:
pipelines = [
    "models/content-engine/__pipeline__.ipynb",
    "models/growth-engine/__pipeline__.ipynb",
    "models/sales-engine/__pipeline__.ipynb",
]

# Execute notebook
for file in tqdm(pipelines):
    print(f"File: {file}")
    print()
    %run $file
    clear_output()

### Get plugins URL

In [ ]:
data = []
plugins = glob.glob(f"{output_dir_plugin}/*.json")

for plugin in plugins:
    plugin_url = naas.asset.find(plugin)
    data.append(
        {
            "name": plugin.split("/")[-1].replace(".json", ""),
            "image": "",
            "url": plugin_url,
            "new_chat_url": f"https://naas.ai/chat/use?plugin_url={plugin_url}"
        }
    )
    
data

### Schedule pipelines

#### 📲 Content Engine

In [ ]:
# Init
engine_name = "content-engine"
engine_model_dir = os.path.join(naas_data_product.MODELS_PATH, engine_name)

# Schedule pipeline
file_path = os.path.join(engine_model_dir, "__pipeline__.ipynb")
cron = "0 8 * * *"
print("⏰ Scheduler:", file_path)
naas.scheduler.add(file_path, cron=cron)
print()

# Uncomment the line below to delete your automation
# naas.scheduler.delete(file_path)

# Add dependencies
files = glob.glob(os.path.join(engine_model_dir, "pipeline_templates", "*.ipynb")) + [os.path.join(engine_model_dir, "__plugin__.ipynb")]
for file in files:
    print("🔗 Dependency:", file)
    naas.dependency.add(file)
    print()
                  
    # Uncomment the line below to delete your dependencies
    # naas.dependency.delete(file)

#### 🚀 Growth Engine

In [ ]:
# Init
engine_name = "growth-engine"
engine_model_dir = os.path.join(naas_data_product.MODELS_PATH, engine_name)

# Schedule pipeline
file_path = os.path.join(engine_model_dir, "__pipeline__.ipynb")
cron = "15 8 * * *"
print("⏰ Scheduler:", file_path)
naas.scheduler.add(file_path, cron=cron)
print()

# Uncomment the line below to delete your automation
# naas.scheduler.delete(file_path)

# Add dependencies
files = glob.glob(os.path.join(engine_model_dir, "pipeline_templates", "*.ipynb")) + [os.path.join(engine_model_dir, "__plugin__.ipynb")]
for file in files:
    print("🔗 Dependency:", file)
    naas.dependency.add(file)
    print()
                  
    # Uncomment the line below to delete your dependencies
    # naas.dependency.delete(file)

#### ⚡️ Sales Engine

In [ ]:
# Init
engine_name = "sales-engine"
engine_model_dir = os.path.join(naas_data_product.MODELS_PATH, engine_name)

# Schedule pipeline
file_path = os.path.join(engine_model_dir, "__pipeline__.ipynb")
cron = "0 9 * * *"
print("⏰ Scheduler:", file_path)
naas.scheduler.add(file_path, cron=cron)
print()

# Uncomment the line below to delete your automation
# naas.scheduler.delete(file_path)

# Add dependencies
files = glob.glob(os.path.join(engine_model_dir, "pipeline_templates", "*.ipynb")) + [os.path.join(engine_model_dir, "__plugin__.ipynb")]
for file in files:
    print("🔗 Dependency:", file)
    naas.dependency.add(file)
    print()
                  
    # Uncomment the line below to delete your dependencies
    # naas.dependency.delete(file)

### Add utils to dependencies

In [ ]:
import naas
import glob
import naas_data_product

files = glob.glob(f"{naas_data_product.UTILS_PATH}/*.ipynb")

for file in files: 
    naas.dependency.add(file)
    
#     # Uncomment the line below to delete your dependencies
#     naas.dependency.delete(file)